In [ ]:
from mlflow.tracking import MlflowClient
import mlflow

MFLOW_TRACKING_URI="sqlite:///mlflow.db"

client = MlflowClient(MFLOW_TRACKING_URI)

2025/08/09 11:26:49 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/08/09 11:26:49 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [2]:
client.search_experiments()
client.create_experiment("my-cool-experiment")

MlflowException: Experiment(name=my-cool-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-cool-experiment', None, 'active', 1754735210923, 1754735210923)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [7]:
client.search_experiments()

[<Experiment: artifact_location='file:///c:/Users/henri/Desktop/Ideias/MLOps/zoomcamp/mlops-zoomcamp/mlruns/2', creation_time=1754419688749, experiment_id='2', last_update_time=1754419688749, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/henri/Desktop/Ideias/MLOps/zoomcamp/mlops-zoomcamp/mlruns/1', creation_time=1754160086452, experiment_id='1', last_update_time=1754160086452, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [3]:
from mlflow.entities import ViewType


runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [16]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse:{run.data.metrics['rmse']}, model: {run.info.artifact_uri}")

run id: 433d2394a29946068e1149034bb3ad61, rmse:6.453424917727452, model: file:///c:/Users/henri/Desktop/Ideias/MLOps/zoomcamp/mlops-zoomcamp/mlruns/1/433d2394a29946068e1149034bb3ad61/artifacts
run id: 7ac1b56e0e684d7b8ebea171c8e103a9, rmse:6.474643167015394, model: file:///c:/Users/henri/Desktop/Ideias/MLOps/zoomcamp/mlops-zoomcamp/mlruns/1/7ac1b56e0e684d7b8ebea171c8e103a9/artifacts
run id: 782ddc76d3c44729b1df7019bf28e84b, rmse:6.474643167015394, model: file:///c:/Users/henri/Desktop/Ideias/MLOps/zoomcamp/mlops-zoomcamp/mlruns/1/782ddc76d3c44729b1df7019bf28e84b/artifacts


In [10]:
model_name = "xgboost"
latest_versions = client.get_latest_versions(name = model_name)
for version in latest_versions:
    print(f"{version.aliases}")

['staging']


C:\Users\henri\AppData\Local\Temp\ipykernel_16148\1633866835.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name = model_name)


In [9]:
client.set_registered_model_alias(
    name = model_name,
    alias = "prod",
    version='1'
)

In [12]:
import mlflow

mlflow.set_tracking_uri(MFLOW_TRACKING_URI)

In [ ]:
mlflow.get_logged_model

In [20]:

run_id = "433d2394a29946068e1149034bb3ad61"
model_uri = f"runs:/{run_id}/models"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2025/08/09 11:48:32 WARNING mlflow.tracking._model_registry.fluent: Run with id 433d2394a29946068e1149034bb3ad61 has no artifacts at artifact path 'models', registering model based on models:/m-fd4f34a06bbf457d9afa62e1209cd8c6 instead
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1754736512392, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1754736512392, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='433d2394a29946068e1149034bb3ad61', run_link=None, source='models:/m-fd4f34a06bbf457d9afa62e1209cd8c6', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [22]:
model_name = "nyc-taxi-regressor"
model_version = 2
new_alias = "staging"
client.set_registered_model_alias(
    name=model_name,
    version=model_version,
    alias = new_alias
)

In [23]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}"
)

<ModelVersion: aliases=['staging'], creation_timestamp=1754736512392, current_stage='None', deployment_job_state=None, description='The model version 2 was transitioned to staging on 2025-08-09', last_updated_timestamp=1754736786684, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='433d2394a29946068e1149034bb3ad61', run_link=None, source='models:/m-fd4f34a06bbf457d9afa62e1209cd8c6', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [28]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [30]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [31]:

client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'c:\\Users\\henri\\Desktop\\Ideias\\MLOps\\zoomcamp\\mlops-zoomcamp\\preprocessor'

In [32]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [33]:
X_test = preprocess(df, dv)

In [34]:
target = "duration"
y_test = df[target].values

In [35]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)


CPU times: total: 0 ns
Wall time: 8.07 ms


MlflowException: No versions of model with name 'nyc-taxi-regressor' and stage 'Production' found

In [ ]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)


In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)